In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import utils
import pandas as pd
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import os
from PIL import Image
Image.MAX_IMAGE_PIXELS = 1000000000

Using TensorFlow backend.


In [2]:
import keras
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, MaxPooling2D, Flatten
from keras.regularizers import l2

In [ ]:
# train_data_path = '/home/ec2-user/code/left-pca16' # Full path should be given here 
# train_mask_path = 'mask/train/'
# test_data_path = '/home/ec2-user/code/right-pca16' # Full path should be given here 
# test_mask_path = 'mask/test/'

## Visualize Masks

In [ ]:
files = []
folder = 'mask/train/'
path = []
for filename in os.listdir('mask/train/'):
    files.append(filename)
    path.append('{}{}'.format(folder,filename))

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.ravel()
for i in range(len(path)):
    img = load_img(path=path[i],grayscale=True, target_size=(150,150), interpolation='nearest')
    axs[i].imshow(img)
    axs[i].set_title(files[i])

In [ ]:
files = []
folder = 'mask/test/'
path = []
for filename in os.listdir('mask/test/'):
    files.append(filename)
    path.append('{}{}'.format(folder,filename))

In [ ]:
fig, axs = plt.subplots(2, 3, figsize=(12, 8))
axs = axs.ravel()
for i in range(len(path)):
    img = load_img(path=path[i],grayscale=True, target_size=(150,150), interpolation='nearest')
    axs[i].imshow(img)
    axs[i].set_title(files[i])

In [ ]:
# Read data form training and test 

In [9]:
train_data_path = '/Users/mlotfollahi/Desktop/Galvanize/Capstone_Project/code/left-pca16' # Full path should be given here 
train_mask_path = 'mask/train/'

test_data_path = '/Users/mlotfollahi/Desktop/Galvanize/Capstone_Project/code/right-pca16' # Full path should be given here 
test_mask_path = 'mask/test/'

In [4]:
X_train,Y_train, num_bands = utils.load_data(data_path = train_data_path, 
                                             masks_path = train_mask_path, 
                                             crop_size = 33, 
                                             num_classes = 6, 
                                             samples = 1000, 
                                             balance = True)



 balance:  True

 number of samples:  6000 

loading class:  1  - samples:  1000
loading class:  2  - samples:  1000
loading class:  3  - samples:  1000
loading class:  4  - samples:  1000
loading class:  5  - samples:  1000
loading class:  6  - samples:  1000


In [ ]:
# X_test,Y_test, num_bands = utils.load_data(data_path = test_data_path, 
#                                            masks_path = test_mask_path, 
#                                            crop_size = 33, 
#                                            num_classes = 6, 
#                                            samples = 100000, 
#                                            balance = True)

In [ ]:
# creat generator

In [5]:
def DataGenerator(data_path, masks_path, crop_size, num_classes, samples = 1000, balance = True):  
    while True:
        X, Y, num_bands = utils.load_data(data_path, 
                                          masks_path, 
                                          crop_size, 
                                          num_classes, 
                                          samples,
                                          balance)
        X_for_keras = np.array([X[i] for i in range(X.shape[0])])
        Y_for_keras = np.array([Y[i,:] for i in range(Y.shape[0])])
        yield X_for_keras, Y_for_keras

In [12]:
train_generator = DataGenerator(data_path = train_data_path, 
                                   masks_path = train_mask_path, 
                                   crop_size = 33, 
                                   num_classes = 6, 
                                   samples = 10, 
                                   balance = True,
                                   num_of_batches = 10)


validation_generator = DataGenerator(data_path = test_data_path, 
                               masks_path = test_mask_path, 
                               crop_size = 33, 
                               num_classes = 6, 
                               samples = 10, 
                               balance = True,
                               num_of_batches = 10)

In [ ]:
print(type(X_train), type(Y_train))
print(X_train.shape, Y_train.shape, num_bands)
print(X_test.shape,  Y_test.shape,  num_bands)

In [ ]:
X_train_list = np.array([X_train[i] for i in range(X_train.shape[0])])

In [ ]:
X_test_list = np.array([X_test[i] for i in range(X_test.shape[0])])

In [ ]:
Y_train_list = np.array([Y_train[i,:] for i in range(Y_train.shape[0])])

In [ ]:
Y_test_list = np.array([Y_test[i,:] for i in range(Y_test.shape[0])])

# Build First Model

In [13]:
# model = Sequential()
# model.add(Conv2D(32, (3, 3), input_shape=(33, 33, 16)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(32, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(64, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(6))
# model.add(Activation('softmax'))

model = Sequential()
model.add(Conv2D(filters=8, kernel_size=(5, 5), strides=(1, 1),
                 activation='relu',
                 input_shape=(33, 33, 16),
                 kernel_regularizer=l2(0.01)))

model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2, 2)))

model.add(Conv2D(filters=32, kernel_size=(5, 5),
                 strides=(1, 1),
                 activation='relu',
                 kernel_regularizer=l2(0.01)))

model.add(MaxPooling2D(pool_size=(2, 2),
                       strides=(2, 2)))
model.add(Flatten())
model.add(Dense(128,
                activation='relu',
                kernel_regularizer=l2(0.01)))
model.add(Dense(6,
                activation='softmax',
                kernel_regularizer=l2(0.01)))

In [14]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy'])

In [15]:
model.fit_generator(
    train_generator,
    epochs = 10,
    validation_data=validation_generator)

ValueError: `steps_per_epoch=None` is only valid for a generator based on the `keras.utils.Sequence` class. Please specify `steps_per_epoch` or use the `keras.utils.Sequence` class.

In [ ]:
# model.fit(X_train_list, Y_train_list,
#           epochs = 5,
#           verbose = 1,
#           batch_size= 50,
#           validation_data=(X_test_list, Y_test_list))

In [ ]:
# # read data from S3
# import boto3
# client = boto3.client('s3') 
# #s3 = boto3.resource('s3')
# client.list_objects(Bucket=s3_bucket, Prefix = "{}/".format(subdirectory), Delimiter='/')['Contents']
# s3_bucket = 'capstonelotfollahi'
# subdirectory = 'left-pca16'
# s3.list_objects(Bucket=s3_bucket, Prefix = "{}/".format(subdirectory), Delimiter='/')["Contents"]

In [ ]:
#  # Build and return a list
# def firstn(n):
#     num, nums = 0, []
#     while num < n:
#         nums.append(num)
#         num += 1     
#     return nums
# sum_of_first_n = sum(firstn(1000000))

In [ ]:
# def createGenerator():
#     mylist = range(3)
#     for i in mylist:
#         yield i*i, i*2

In [ ]:
# gen = createGenerator()